#### Python Memory Management
Memory management in Python involves a combination of automatic garbage collection, reference counting, and various internal optimizations to efficiently manage memory allocation and deallocation. Understanding these mechanisms can help developers write more efficient and robust applications.

1. Key Concepts in Python Memory Management
2. Memory Allocation and Deallocation
3. Reference Counting
4. Garbage Collection
5. The gc Module
6. Memory Management Best Practices

#### Reference Counting
Reference counting is the primary method Python uses to manage memory. Each object in Python maintains a count of references pointing to it. When the reference count drops to zero, the memory occupied by the object is deallocated.

In [1]:
import sys

a=[]
## 2 (one reference from 'a' and one from getrefcount())
print(sys.getrefcount(a))

2


In [2]:
b=a
print(sys.getrefcount(b))

3


In [3]:
del b
print(sys.getrefcount(a))

2


In [23]:
del a
a = [True]
print(sys.getrefcount(a))


2


#### Garbage Collection
Python includes a cyclic garbage collector to handle reference cycles. Reference cycles occur when objects reference each other, preventing their reference counts from reaching zero.

In [13]:
import gc
## enable garbage collection
gc.enable()



In [14]:
gc.disable()

In [15]:
gc.collect()

1196

In [47]:
### Get garbage collection stats
print(gc.get_stats())

[{'collections': 188, 'collected': 5304, 'uncollectable': 0}, {'collections': 16, 'collected': 310, 'uncollectable': 0}, {'collections': 10, 'collected': 15023, 'uncollectable': 0}]


In [48]:
### get unreachable objects
print(gc.garbage)

[]


####  Memory Management Best Practices
1. Use Local Variables: Local variables have a shorter lifespan and are freed sooner than global variables.
2. Avoid Circular References: Circular references can lead to memory leaks if not properly managed.
3. Use Generators: Generators produce items one at a time and only keep one item in memory at a time, making them memory efficient.
4. Explicitly Delete Objects: Use the del statement to delete variables and objects explicitly.
5. Profile Memory Usage: Use memory profiling tools like tracemalloc and memory_profiler to identify memory leaks and optimize memory usage.

In [49]:
## Handled Circular reference
import gc

class MyObject:
    def __init__(self, name):
        self.name = name
        print(f"Object {self.name} created")

    def __del__(self):
        print(f"Object {self.name} deleted")

    def __str__(self):
        return f"Printing Object: {self.name}"

# Create circular reference
obj1 = MyObject("obj1")
obj2 = MyObject("obj2")
obj1.ref = obj2
obj2.ref = obj1

print(obj1)
print(obj2)

del obj1
del obj2

# ## Manually trigger the garbage collection
gc.collect()



Object obj1 created
Object obj2 created
Printing Object: obj1
Printing Object: obj2
Object obj1 deleted
Object obj2 deleted


10

In [44]:
gc.disable()
print(gc.garbage)

gc.enable()
gc.collect()

[]
Object obj1 deleted
Object obj2 deleted
Object obj1 deleted
Object obj2 deleted


263

In [50]:
## Generators For Memory Efficiency
#Generators allow you to produce items one at a time, using memory efficiently by only keeping one item in memory at a time.

def generate_numbers(n):
    for i in range(n):
        yield i

## using the generator
for num in generate_numbers(100000):
    print(num)
    if num>10:
        break

0
1
2
3
4
5
6
7
8
9
10
11


In [51]:
## Profiling Memory USage with tracemalloc
import tracemalloc

def create_list():
    return [i for i in range(10000)]

def main():
    tracemalloc.start()
    
    create_list()
    
    snapshot = tracemalloc.take_snapshot()
    top_stats = snapshot.statistics('lineno')
    
    print("[ Top 10 ]")
    for stat in top_stats[::]:
        print(stat)


In [52]:
main()

[ Top 10 ]
/opt/anaconda3/envs/ml_venv/lib/python3.12/tokenize.py:570: size=15.1 KiB, count=275, average=56 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/IPython/core/builtin_trap.py:77: size=6512 B, count=1, average=6512 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/zmq/sugar/attrsettr.py:45: size=4747 B, count=101, average=47 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/codeop.py:126: size=4732 B, count=57, average=83 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/traitlets/traitlets.py:731: size=4216 B, count=68, average=62 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/asyncio/selector_events.py:132: size=4129 B, count=1, average=4129 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/IPython/core/compilerop.py:174: size=4027 B, count=56, average=72 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/traitlets/traitlets.py:1514: size=3360 B, count=28, average=120 B
/opt/anaconda3/envs/ml_venv/lib/python3.12/site-packages/traitlets/

In [53]:
gc.collect()

0